In [1]:
import thingspeak
import pandas as pd
import json
import ssl
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio
import datetime
from sklearn.model_selection import train_test_split
import math
import seaborn as sn
from typing import Tuple

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

ssl._create_default_https_context = ssl._create_unverified_context



In [2]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

from datetime import datetime, timedelta
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sbn

# Configuring Matplotlib
import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 300
savefig_options = dict(format="png", dpi=300, bbox_inches="tight")

from scipy.ndimage.filters import gaussian_filter

In [3]:
API_KEY = "ZRDS32VNQEEFSOF4"
CHANNEL_ID = "1361623"

fields = {
    "field2": "TEMP [C]", 
    "field3":"Relative humidity", 
    "field4":"PM1 [ug/m3]", 
    "field5":"PM2.5 [ug/m3]", 
    "field6":"PM10 [ug/m3]",
    "created_at" : "Date/time"
}

new_fields = {v : k for (k,v) in zip(fields.keys(), fields.values())}

In [4]:
def setup_plotly():
    pd.options.plotting.backend = 'plotly'

setup_plotly()


In [5]:
def get_thingspeak_data(id: str, api_key: str, nb_of_results: int = 8000, drop_entry_id: bool = True, save: bool = False) -> pd.DataFrame:
    time_zone = "UTC"
    #url = f"https://thingspeak.com/channels/{id}/feed.csv?apikey={api_key}&results={nb_of_results}&timezone={time_zone}"
    
    #df = pd.read_csv(url)
    df = pd.read_csv('data/thingspeak_data.csv')

    df.replace(['None'], np.nan, inplace=True)
    df.rename(columns=fields, inplace=True)
    df["Date/time"] = pd.to_datetime(df["Date/time"])
    df["TEMP [C]"] = df["TEMP [C]"].astype("float32")
    df.set_index("Date/time", inplace=True)
    df.index = df.index.tz_convert("Europe/Paris")
    if save:
        df.to_csv('data/thingspeak_data.csv')
    df = df.assign(missing= np.nan)
    df.drop("missing", inplace=True, axis=1)
    df.drop("Relative humidity", axis=1, inplace=True)
    df.drop("PM1 [ug/m3]", axis=1, inplace=True)
    
    if drop_entry_id:
        df.drop('entry_id', axis=1, inplace=True)


    return df

In [6]:
def get_zue_data():
    df = pd.read_csv('data/ZUE.csv', delimiter=";")
    df["Date/time"] = pd.to_datetime(df["Date/time"], format='%d.%m.%Y %H:%M')
    df["TEMP [C]"] = df["TEMP [C]"].astype("float32")
    df.set_index("Date/time", inplace=True)
    df.index = df.index.tz_localize("Europe/Paris", ambiguous="NaT", nonexistent="shift_backward")
    df = df.assign(missing= np.nan)
    df.drop("missing", inplace=True, axis=1)

    return df

In [7]:
def extract_time_series(df: pd.DataFrame):
    pm10_series = df.loc[:, "PM10 [ug/m3]"]
    pm25_series = df.loc[:, "PM2.5 [ug/m3]"]
    temperature_series = df.loc[:, "TEMP [C]"]
    precipitation_series = df.loc[:, "PREC [mm]"]

    return temperature_series, precipitation_series, pm25_series, pm10_series

In [8]:
# we use a sliding window with std to detect and remove outliers
def remove_outliers(df, param=1.6, hours=0, minutes=0):
    for t in df.index:
        t1 = t - timedelta(hours=hours, minutes=minutes)
        t2 = t + timedelta(hours=hours, minutes=minutes)
        inter = df.loc[t1:t2]
        mean = inter.mean(axis=0)
        std = inter.std(axis=0)
        dist = np.abs(df.loc[t] - mean)
        df.loc[t] = df.loc[t].where(dist < param * std)

    return df

In [9]:
# we wanted to interpolate the large missing intervals with DTW but the results were inconclusive
# due to the high correlation between pm10 and pm25 (~0.95) we decided to interpolate with a simple multiplication
def interpolate_pm10(df):
    pm10_series = df["PM10 [ug/m3]"]
    pm25_series = df["PM2.5 [ug/m3]"]

    mean_factor = (pm10_series/pm25_series).mean(axis=0)

    # interpolate with multiplication of pm2.5
    pm10_na = pm10_series.isna()
    pm10_series[pm10_na] = mean_factor * pm25_series[pm10_na]

    df["PM10 [ug/m3]"] =  pm10_series

    return df

In [77]:
def merge_datasets(df_zue: pd.DataFrame, df_ts: pd.DataFrame, save_data=False) -> pd.DataFrame:

    # time interpolation corresponds to a linear interpolation but 
    # it takes into account the time distance between each index when
    # assigning a value

    # except for the case of pm10 we decided to use time interpolation for 
    # scattered missing values

    # reorder the columns according to zue data
    # crop the ts data that has no corresponding zue data
    # resample the ts data to have exactly 5 minutes indexes
    ts = df_ts.copy()
    ts = ts[ts.index.notnull()]
    ts = ts[df_zue.columns[:3]]
    ts = ts.loc[:df_zue.index[-1]] 
    ts = ts.resample('5min').mean()
    start = ts.index[0]

    if save_data : ts.to_csv('data_v/ts.csv')
 
    # crop zue data and save precipitations separately
    zue = df_zue.copy()
    zue = zue[zue.index.notnull()]
    zue = zue.loc[:ts.index[-1]]
    precipitations = zue["PREC [mm]"]
    zue.drop("PREC [mm]", axis=1, inplace=True)

    if save_data : zue.to_csv('data_v/zue.csv')

    # interpolate the pm10 zue data for which we have large missing intervals
    zue = interpolate_pm10(zue)

    if save_data : zue.to_csv('data_v/zue_pm10_inter.csv')

    # remove the outliers and interpolate their value with time
    zue = remove_outliers(zue, hours=5)
    ts = remove_outliers(ts, minutes=30)

    zue = zue.interpolate(method='time')
    ts = ts.interpolate(method='time')

    if save_data : zue.to_csv('data_v/zue_no_outliers.csv')
    if save_data : ts.to_csv('data_v/ts_no_outliers.csv')
   
    # upsample zue data to 5mn and interpolate with time
    # separate between previous and overlapping data
    zue = zue.resample('5min').interpolate('time')
    zue_prev = zue.loc[:start].drop(start)
    zue_superpos = zue.loc[start:ts.index[-1]]

    if save_data : zue_superpos.to_csv('data_v/zue_superpos.csv')

    # scale our ts data according to the zue data
    h_mean_ts = pd.DataFrame().reindex_like(ts)
    for t in ts.index:
        t1 = t - timedelta(minutes=30)
        t2 = t + timedelta(minutes=30)
        h_mean_ts.loc[t] = ts.loc[t1:t2].mean(axis=0)

    if save_data : h_mean_ts.to_csv('data_v/h_mean_ts.csv')

    scaled_ts = zue_superpos * ts / h_mean_ts

    if save_data : scaled_ts.to_csv('data_v/scaled_ts.csv')
    
    # calculate our scaled ts data variation around the zue data
    dist_ts = scaled_ts - zue_superpos

    #h = dist_ts.hist()
    #h.show()
    
    # calculate our variation properties (assumed normal from the histogram)
    means = dist_ts.mean(axis = 0)
    covs = dist_ts.cov()

    # add noise to the zue data with a distribution corresponding to the ts data
    zue_prev += np.random.multivariate_normal(means, covs, zue_prev.shape[0])

    # create the result dataset by concatenating the noisy previous zue data with
    # the scaled ts data, adding back precipitations, interpolating any possibly missing values and 
    # setting the minimal value for the particule measures to 0
    result = pd.concat([zue_prev, scaled_ts])
    result["PREC [mm]"] = precipitations
    result = result.interpolate('time')
    result[["PM10 [ug/m3]", "PM2.5 [ug/m3]"]] = result[["PM10 [ug/m3]", "PM2.5 [ug/m3]"]].clip(lower=0)

    if save_data : result.to_csv('data_v/dataset.csv')

    return result

In [78]:
def get_dataset(save: bool = True) -> pd.DataFrame: 
    df_zue = get_zue_data()
    df_ts = get_thingspeak_data(id=CHANNEL_ID, api_key=API_KEY, save=True) 
    df = merge_datasets(df_ts=df_ts, df_zue=df_zue)
    if save :
        result.to_csv('data/dataset.csv')
    return df

In [65]:
df = get_dataset(save=False)

ipykernel_launcher:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
df_ts = get_thingspeak_data(id=CHANNEL_ID, api_key=API_KEY, save=True)
df_ts.index = df_ts.index.round('5T')
df_ts = df_ts.groupby(level=0).mean().resample('5min').mean()
df_ts2 = df_ts.resample('5min').mean()
print(np.where(df_ts != df_ts2))
df.plot(title="Comparison graph", width=1100)

(array([  18,   18,   18,  122,  122,  122,  144,  144,  144,  174,  174,
        174,  349,  351,  355,  360,  361,  362,  364,  366,  375,  376,
        552,  552,  552,  634,  634,  634,  750,  750,  750,  860,  860,
        860,  861,  861,  861,  862,  862,  862,  863,  863,  863, 1400,
       1400, 1400, 1521, 1521, 1521, 1588, 1588, 1588], dtype=int64), array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0,
       1, 2, 0, 1, 2, 0, 1, 2], dtype=int64))


In [ ]:
df1 = pd.read_csv('data_v/ts_no_outliers.csv')
df1.set_index("Date/time", inplace=True)
df1.columns = df1.columns + ' TS'

df2 = pd.read_csv('data_v/zue_superpos.csv')
df2.set_index("Date/time", inplace=True)
df2.columns = df2.columns + ' Zue'

df3 = pd.read_csv('data_v/h_mean_ts.csv')
df3.set_index("Date/time", inplace=True)
df3.columns = df3.columns + ' TS Mean'

df4 = pd.read_csv('data_v/scaled_ts.csv')
df4.set_index("Date/time", inplace=True)
df4.columns = df4.columns + ' TS Scaled'

dfplot = df1.append(df2).append(df3).append(df4)

dfplot.plot(title="Comparison graph", width=1100)

In [ ]:
dfplot = pd.read_csv('data_v/dataset.csv')
dfplot.set_index("Date/time", inplace=True)
dfplot = dfplot.applymap(lambda x : max(x, 0))
dfplot.plot(title="Comparison graph", width=1100)

In [ ]:
temperature_series, precipitation_series, pm25_series, pm10_series = extract_time_series(df)

In [ ]:
fig = df.plot(title='Dataset')
fig.update_layout(width=1100)
fig.show()

In [ ]:
corrMatrix = df.corr()
px.imshow(corrMatrix)

In [ ]:
plt = sn.heatmap(corrMatrix, annot=True)
plt.get_figure().savefig('plots/correlogram.png')

In [ ]:
precipitation_series.plot(title="Precipitation time-series", width=1100)

In [ ]:
temperature_series.plot(title="Temperature time-series", width=1100)

In [ ]:
pm10_series.plot(title="PM10 time-series", width=1100)

In [ ]:
pm25_series.plot(title="PM2.5 time-series", width=1100)

In [ ]:
#initial time series
x = pm10_series.to_numpy()
y = pm25_series.to_numpy()

In [ ]:
#blur the ts
x = gaussian_filter(x, sigma=7)
y = gaussian_filter(y, sigma=7)

In [ ]:
#reduce the ts to intervals
x = x[2000:3000]
y = y[2000:3000]

In [ ]:
dtw_distance, warp_path = fastdtw(x, y, dist=3)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))

# Remove the border and axes ticks
fig.patch.set_visible(False)
ax.axis('off')

#for [map_x, map_y] in warp_path[::10]:
#    ax.plot([map_x, map_y], [x[map_x], y[map_y]], '-k')

ax.plot(x, color='blue', linewidth=1)
ax.plot(y, color='red', linewidth=1)
#ax.tick_params(axis="both", which="major", labelsize=18)

fig.savefig("plots/comparison_with_div_inter.png", **savefig_options)

In [ ]:
def compare_imputation_methods(df: pd.DataFrame)

    pass

In [ ]:
def calc_percent_NAs(df: pd.DataFrame):
    nans = pd.DataFrame(df.isnull().sum().sort_values(ascending=False)/len(df), columns=['percent']) 
    idx = nans['percent'] > 0
    return nans[idx]

In [ ]:
calc_percent_NAs(df)

In [ ]:
def split_into_train_test_validation_sets(X: np.array) -> Tuple[np.array, np.array, np.array]:
    train_size = int(0.7 * len(X))
    validation_size = int(0.1 * len(X))
    test_size = len(X) - validation_size - train_size

    train_set, test_set, validation_set = X[0:train_size, :], X[train_size: train_size + test_size,:], X[train_size + test_size: train_size + test_size + validation_size, :]
    
    print(f"""
    Total dataset length: {len(X)}
    
    Train set shape: {train_set.shape} ({train_set.shape[0] / len(X) * 100:0.2f}% of dataset)
    Test set shape: {test_set.shape} ({test_set.shape[0] / len(X) * 100:0.2f}% of dataset)
    Validation set shape: {validation_set.shape} ({validation_set.shape[0] / len(X) * 100:0.2f}% of dataset)
    """)

    return train_set, test_set, validation_set

In [ ]:
def normalize_dataset(df: pd.DataFrame) -> np.array:
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(df)

In [ ]:
scaled_data = normalize_dataset(df=df)
train_set, test_set, validation_set = split_into_train_test_validation_sets(X=scaled_data)